In [2]:
import pandas as pd
from math import floor
from idaes.core.util import model_statistics as mstat
import pyomo.environ as pyo
import numpy as np

In [3]:
def data_preprocessing(df, element_list):

    # Remove Nans
    df = df.dropna()
    df.reset_index(drop=True)

    # # Remove outliers based on previewing data
    # for s in outlier_list:
    #     df = df.drop(index=s - 2)

    # df = df.reset_index(drop=True)

    # add mmols column
    for e in element_list:
        for i in df.index.to_list():
            df[f"[{e}] (mmol/L)"] = df[f"[{e}] (mol/L)"] * 1000

    # normalizing and creating sf dictionary
    sf_dict = {}
    scaled_cols = {}
    for col in df.columns:
        scaled_column_name = f"{col}_scaled"
        if df[col].dtype in ["float64", "int64"]:
            if col in [f"[{e}] (mmol/L)" for e in element_list]:
                max_value = abs(df[col]).max()
                scaled_cols[scaled_column_name] = df[col] / max_value
            else:
                max_value = 1
                scaled_cols[scaled_column_name] = df[col] / max_value

            sf_dict[col] = max_value
            # print(f"Scaled {col} by {max_value}")
    df = pd.concat([df, pd.DataFrame(scaled_cols)], axis=1)

    # get scaling factors

    scaling_factor = {}
    for e in element_list:
        scaling_factor[e] = sf_dict[f"[{e}] (mmol/L)"]

    return df, sf_dict, scaling_factor


# df, sf_dict, scaling_factor = data_preprocessing(df, [3, 11, 23, 25, 29, 31], Elements, 1000)

In [4]:

def regression_model_development(df, element_list, scaling_factor):

    n_data = df.shape[0]

    m = pyo.ConcreteModel()

    m.I = pyo.RangeSet(0, n_data - 1)
    m.element_set = pyo.Set(initialize=element_list)
    m.f_param_index = pyo.Set(initialize=element_list + ["constant"])

    # Input variables

    m.pH = pyo.Var(m.I, domain=pyo.Reals, initialize=0.0)
    m.C_ex = pyo.Var(m.I, domain=pyo.Reals, initialize=0.0)
    m.Cfeed = pyo.Var(m.I, m.element_set, domain=pyo.Reals, initialize=0.0)

    # Output variable

    m.logD = pyo.Var(m.I, m.element_set, domain=pyo.Reals, initialize=0.0)

    # Model variables

    m.logD_scaled_pred = pyo.Var(m.I, m.element_set, initialize=0.1, domain=pyo.Reals)
    m.Cext_net_pred = pyo.Var(m.I, domain=pyo.Reals, initialize=0.1, bounds=(1e-5, 2))
    m.logCext_net_pred = pyo.Var(
        m.I, domain=pyo.Reals, initialize=0.1, bounds=(np.log10(1e-10), np.log10(2))
    )

    # Parameters

    ai_list = pyo.RangeSet(2)

    m.ai = pyo.Var(ai_list, m.element_set, domain=pyo.Reals, initialize=0.5)
    m.bi = pyo.Var(ai_list, m.element_set, domain=pyo.Reals, initialize=0.5)
    m.ci = pyo.Var(m.element_set, m.element_set, domain=pyo.Reals, initialize=0.5)
    m.di = pyo.Var(m.element_set, m.element_set, domain=pyo.Reals, initialize=0.0)
    m.fi = pyo.Var(m.f_param_index, domain=pyo.NonNegativeReals, initialize=0.5)

    m.alpha = pyo.Var(m.I, m.element_set, domain=pyo.Reals, initialize=0.5)
    m.beta = pyo.Var(m.I, m.element_set, domain=pyo.Reals, initialize=0.5)

    m.ai_abs = pyo.Var(
        ai_list, m.element_set, domain=pyo.NonNegativeReals, initialize=0.5
    )
    m.bi_abs = pyo.Var(
        ai_list, m.element_set, domain=pyo.NonNegativeReals, initialize=0.5
    )
    m.ci_abs = pyo.Var(
        m.element_set, m.element_set, domain=pyo.NonNegativeReals, initialize=0.5
    )
    m.di_abs = pyo.Var(
        m.element_set, m.element_set, domain=pyo.NonNegativeReals, initialize=0.0
    )
    m.fi_abs = pyo.Var(m.f_param_index, domain=pyo.NonNegativeReals, initialize=0.5)

    # Initialize input variables

    name_mapper_general = {
        "pH_scaled": m.pH,
        "[D2EHPA]_scaled": m.C_ex,
    }

    def map_generator(element):
        return {
            f"[{element}] (mmol/L)_scaled": m.Cfeed,
            f"log D (obs)_{element}_scaled": m.logD,
        }

    name_mapper_elements = {}

    for element in m.element_set:
        name_mapper_elements[element] = map_generator(element)

    for col in name_mapper_general.keys():
        for i in df.index:
            name_mapper_general[col][i].fix(df[col][i])

    for element in m.element_set:
        for i in df.index:
            for col in name_mapper_elements[element].keys():
                name_mapper_elements[element][col][i, element].fix(df[col][i])

    # Data Fixing

    def unique_element_counter(reference_element, contribution_element, df):
        reference_element_set = df[df[f"weight {reference_element}_scaled"] == 1]
        contribution_wrt_reference_set = reference_element_set[
            reference_element_set[f"[{contribution_element}] (mol/L)_scaled"] != 0
        ]
        unique_values = len(
            contribution_wrt_reference_set[
                f"[{contribution_element}] (mol/L)_scaled"
            ].unique()
        )
        return unique_values

    FV = pd.DataFrame(index=m.element_set, columns=m.element_set)
    MP = pd.DataFrame(index=m.element_set, columns=m.element_set)
    LC = pd.DataFrame(index=m.element_set, columns=m.element_set)
    QC = pd.DataFrame(index=m.element_set, columns=m.element_set)

    for e in m.element_set:
        for f in m.element_set:
            FV.loc[e, f] = unique_element_counter(e, f, df)

    for e in m.element_set:
        for f in m.element_set:
            MP.loc[e, f] = min(floor(FV.loc[e, f] / 2), 2)

    for e in m.element_set:
        for f in m.element_set:
            LC.loc[e, f] = min(MP.loc[e, f], 1)

    for e in m.element_set:
        for f in m.element_set:
            QC.loc[e, f] = MP.loc[e, f] - LC.loc[e, f]

    for e in m.element_set:
        for f in m.element_set:
            if LC.loc[e, f] == 0:
                m.ci[f, e].fix(0)
            if QC.loc[e, f] == 0:
                m.di[f, e].fix(0)

    for e in m.element_set:
        if (
            len(df[df[f"[{e}] (mol/L)_scaled"] != 0][f"[{e}] (mol/L)_scaled"].unique())
            == 1
        ):
            m.fi[e].fix(0)

    # fixing for terms after L1 regularization analysis

    for e in ["Zn", "Ca"]:
        m.ai[1, e].fix(1e-8)

    for e in ["Mn", "Fe(2)", "Cd", "Mg", "Al"]:
        m.ai[2, e].fix(1e-8)

    for e in ["Fe(2)", "Cd"]:
        m.bi[1, e].fix(1e-8)

    for e in ["Mn", "Ni", "Cu"]:
        m.ci["Co", e].fix(1e-12)

    for e in ["Ni", "Cu"]:
        m.ci[e, "Co"].fix(1e-12)

    for e in ["Mn", "Cu"]:
        m.di["Co", e].fix(1e-12)

    for e in ["Mn", "Co"]:
        m.di["Ni", e].fix(1e-12)

    for e in ["Mn", "Co", "Ni"]:
        m.di["Cu", e].fix(1e-12)

    # Second reduction

    for e in ["Mn", "Co", "Fe(3)"]:
        m.fi[e].fix(1e-14)

    for e in ["Mn", "Ni"]:
        m.di["Mn", e].fix(1e-12)

    # Constraints

    @m.Constraint(m.I)
    def logCext_net_pred_constraint(m, i):
        return 10 ** (m.logCext_net_pred[i]) == m.Cext_net_pred[i]

    @m.Constraint(m.I, m.element_set)
    def alpha_const(m, i, e):
        linear_term = sum(m.ci[s, e] * m.Cfeed[i, s] for s in m.element_set)
        quad_term = sum(m.di[s, e] * m.Cfeed[i, s] ** 2 for s in m.element_set)

        return (
            m.alpha[i, e]
            - m.ai[1, e]
            - m.ai[2, e] * m.C_ex[i]
            - linear_term
            - quad_term
        ) == 0

    @m.Constraint(m.I, m.element_set)
    def b_const(m, i, e):
        return m.beta[i, e] == m.bi[1, e] + m.bi[2, e] * m.logCext_net_pred[i]

    @m.Constraint(m.I)
    def Cext_net_pred_constraint(m, i):
        linear_term = sum(m.fi[s] * m.Cfeed[i, s] for s in m.element_set)
        return m.Cext_net_pred[i] == m.C_ex[i] - linear_term - m.fi["constant"]

    @m.Constraint(m.I)
    def Cext_net_value_constraint(m, i):
        return m.C_ex[i] - m.Cext_net_pred[i] >= 1e-8

    @m.Constraint(m.I, m.element_set)
    def logD_scaled_pred_constraint(m, i, e):
        return m.logD_scaled_pred[i, e] == m.alpha[i, e] * m.pH[i] + m.beta[i, e]

    @m.Constraint(ai_list, m.element_set)
    def ai_negative_bound(m, i, e):
        if m.ai[i, e].fixed == True:
            m.ai_abs[i, e].fix(1e-9)
            return pyo.Constraint.Skip
        else:
            return m.ai_abs[i, e] >= -m.ai[i, e]

    @m.Constraint(ai_list, m.element_set)
    def ai_positive_bound(m, i, e):
        if m.ai[i, e].fixed == True:
            m.ai_abs[i, e].fix(1e-9)
            return pyo.Constraint.Skip
        else:
            return m.ai_abs[i, e] >= m.ai[i, e]

    @m.Constraint(ai_list, m.element_set)
    def bi_negative_bound(m, i, e):
        if m.bi[i, e].fixed == True:
            m.bi_abs[i, e].fix(1e-9)
            return pyo.Constraint.Skip
        else:
            return m.bi_abs[i, e] >= -m.bi[i, e]

    @m.Constraint(ai_list, m.element_set)
    def bi_positive_bound(m, i, e):
        if m.bi[i, e].fixed == True:
            m.bi_abs[i, e].fix(1e-9)
            return pyo.Constraint.Skip
        else:
            return m.bi_abs[i, e] >= m.bi[i, e]

    @m.Constraint(m.element_set, m.element_set)
    def ci_negative_bound(m, f, e):
        if m.ci[f, e].fixed == True:
            m.ci_abs[f, e].fix(1e-9)
            return pyo.Constraint.Skip
        else:
            return m.ci_abs[f, e] >= -m.ci[f, e]

    @m.Constraint(m.element_set, m.element_set)
    def ci_positive_bound(m, f, e):
        if m.ci[f, e].fixed == True:
            m.ci_abs[f, e].fix(1e-9)
            return pyo.Constraint.Skip
        else:
            return m.ci_abs[f, e] >= m.ci[f, e]

    @m.Constraint(m.element_set, m.element_set)
    def di_negative_bound(m, f, e):
        if m.di[f, e].fixed == True:
            m.di_abs[f, e].fix(1e-9)
            return pyo.Constraint.Skip
        else:
            return m.di_abs[f, e] >= -m.di[f, e]

    @m.Constraint(m.element_set, m.element_set)
    def di_positive_bound(m, f, e):
        if m.di[f, e].fixed == True:
            m.di_abs[f, e].fix(1e-9)
            return pyo.Constraint.Skip
        else:
            return m.di_abs[f, e] >= m.di[f, e]

    @m.Constraint(m.f_param_index)
    def fi_negative_bound(m, f):
        if m.fi[f].fixed == True:
            m.fi_abs[f].fix(1e-9)
            return pyo.Constraint.Skip
        else:
            return m.fi_abs[f] >= -m.fi[f]

    @m.Constraint(m.f_param_index)
    def fi_positive_bound(m, f):
        if m.fi[f].fixed == True:
            m.fi_abs[f].fix(1e-9)
            return pyo.Constraint.Skip
        else:
            return m.fi_abs[f] >= m.fi[f]

    @m.Expression()
    def sse_element(m):
        n_data = len(m.I)
        n_elements = len(m.element_set)
        return sum(
            ((m.logD_scaled_pred[i, e] - m.logD[i, e]) ** 2)
            * df[f"weight {e}_scaled"][i]
            for i in m.I
            for e in m.element_set
        ) / (n_data * n_elements)

    @m.Expression()
    def l1_norm_term(m):
        return (
            sum(
                (m.ai_abs[1, e])
                + (m.ai_abs[2, e])
                + (m.bi_abs[1, e])
                + (m.bi_abs[2, e])
                + sum((m.ci_abs[s, e]) / scaling_factor[s] for s in m.element_set)
                + sum((m.di_abs[s, e]) / scaling_factor[s] ** 2 for s in m.element_set)
                + (m.fi_abs[e])
                for e in m.element_set
            )
            + m.fi_abs["constant"]
        )

    return m

In [5]:
def objective_buildup_function(m, rho):

    m.rho = pyo.Param(initialize=rho, mutable=True)

    @m.Objective(sense=pyo.minimize)
    def objective_function(m):
        return m.sse_element + m.rho * m.l1_norm_term


def model_solution(m):

    solver = pyo.SolverFactory("ipopt_v2")
    solver.options["halt_on_ampl_error"] = "no"
    solver.options["max_iter"] = 500

    print(mstat.degrees_of_freedom(m))

    solver.solve(m, tee=True, symbolic_solver_labels=True)

In [6]:
def total_regression_analysis(
    df_raw, outlier_list, element_list, conversion_factor, rho
):
    df, sf_dict, scaling_factor = data_preprocessing(
        df_raw, outlier_list, element_list, conversion_factor
    )
    m = regression_model_development(df, element_list, scaling_factor)
    objective_buildup_function(m, rho)
    m, results = model_solution(m)
    return m, results

In [7]:
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score


def R_squared_calculation(m, df):
    r_square_value = {}

    for e in m.element_set:
        logD_scaled_pred_eff = [
            m.logD_scaled_pred[i, e]() * df[f"weight {e}_scaled"][i] for i in m.I
        ]
        logD_scaled_eff = [m.logD[i, e]() * df[f"weight {e}_scaled"][i] for i in m.I]
        r_square_value[e] = r2_score(logD_scaled_eff, logD_scaled_pred_eff)

    return r_square_value

In [8]:
import pandas as pd

data_path = "General_dataset_DEHPA.xlsx"

df1 = pd.read_excel(data_path, sheet_name="DaSilva")
df2 = pd.read_excel(data_path, sheet_name="Syensqo")
df3 = pd.read_excel(data_path, sheet_name="Peng")
df4 = pd.read_excel(data_path, sheet_name="Metallurgist")

df_total = pd.concat([df1, df2, df3, df4], ignore_index=True)

Elements = [
    "Mn",
    "Co",
    "Ni",
    "Cu",
    "Zn",
    "Fe(3)",
    "Ca",
    "Fe(2)",
    "Cd",
    "Mg",
    "Al",
]

In [9]:
# Preprocess the data
df, sf_dict, scaling_factor = data_preprocessing(
    df_total, Elements
)

threshold = 0.83

# Make the model
m = regression_model_development(df, Elements, scaling_factor)

# Make the objective function
rho = 0.0312 / 186
objective_buildup_function(m, rho)

# Solve the model
model_solution(m)

# Get the R squared values
r_square_old = R_squared_calculation(m, df)

# Get the elements with lower R squared values and sort them
lower_rsquare_elements = {
    k: r_square_old[k] for k in r_square_old.keys() if r_square_old[k] < threshold
}

# go into loop

# if lower_rsquare_elements is empty, stop the regression analysis
while lower_rsquare_elements != {}:

    # get element having least R square value
    element_having_least_R = [
        k
        for k in lower_rsquare_elements.keys()
        if lower_rsquare_elements[k] == min(v for v in lower_rsquare_elements.values())
    ]
    print(f'element with least R is {element_having_least_R}')

    # remove that data point from database and reperform regression
    print('resolving after removing element')
    df.loc[element_having_least_R, f"weight {element_having_least_R}_scaled"] = 0
    model_solution(m)

    # get the new R square values
    r_square_new = R_squared_calculation(m, df)

    # Compare the R squared values
    if (
        r_square_new[element_having_least_R] - r_square_old[element_having_least_R]
    ) < 0.03:
        print(f'no significant improvement in R value for {element_having_least_R}')
        df.loc[element_having_least_R, f"weight {element_having_least_R}_scaled"] = 1
        print(f"this is the best model possible for {element_having_least_R}")
        del lower_rsquare_elements[element_having_least_R]
    else:
        print(f'significant improvement in R value for {element_having_least_R}, continuing regression')
        r_square_old = r_square_new
        lower_rsquare_elements = {
            k: r_square_old[k] for k in r_square_old.keys() if r_square_old[k] < threshold
        }
    



C:\Users\ad00105\AppData\Local\Temp\1\ipykernel_21384\10811894.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f"[{e}] (mmol/L)"] = df[f"[{e}] (mol/L)"] * 1000
C:\Users\ad00105\AppData\Local\Temp\1\ipykernel_21384\10811894.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f"[{e}] (mmol/L)"] = df[f"[{e}] (mol/L)"] * 1000
C:\Users\ad00105\AppData\Local\Temp\1\ipykernel_21384\10811894.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc

KeyError: "Index '162' is not valid for indexed component 'pH'"